In [1]:
import sys
sys.path.insert(0, '../')


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation

import statsmodels.api as sm

import os


from engine.metric import laplace_log_likelihood
from dataset.build import read_df
from config import cfg
import pydicom as dicom

In [2]:
read_df(cfg)

(      index                    Patient  Weeks   FVC    Percent  Age   Sex  \
 0         0  ID00007637202177411956430     -4  2315  58.253649   79  Male   
 1         1  ID00007637202177411956430      5  2214  55.712129   79  Male   
 2         2  ID00007637202177411956430      7  2061  51.862104   79  Male   
 3         3  ID00007637202177411956430      9  2144  53.950679   79  Male   
 4         4  ID00007637202177411956430     11  2069  52.063412   79  Male   
 ...     ...                        ...    ...   ...        ...  ...   ...   
 1530   1544  ID00426637202313170790466     13  2712  66.594637   73  Male   
 1531   1545  ID00426637202313170790466     19  2978  73.126412   73  Male   
 1532   1546  ID00426637202313170790466     31  2908  71.407524   73  Male   
 1533   1547  ID00426637202313170790466     43  2975  73.052745   73  Male   
 1534   1548  ID00426637202313170790466     59  2774  68.117081   73  Male   
 
      SmokingStatus Source  Confidence  ...  FVC_scld  Percent

In [ ]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.head()

In [ ]:
train.Patient.nunique()

In [ ]:
idx = np.random.choice(train.Patient.unique())

In [ ]:
train[train['Patient']==idx]

In [ ]:
plt.figure()
plt.plot(train.loc[train['Patient']==idx ,'FVC'])

In [ ]:
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

In [ ]:
def set_lungwin(img, hu=[-1200., 600.]):
    lungwin = np.array(hu)
    newimg = (img-lungwin[0]) / (lungwin[1]-lungwin[0])
    newimg[newimg < 0] = 0
    newimg[newimg > 1] = 1
    newimg = (newimg * 255).astype('uint8')
    return newimg

In [ ]:
scans = load_scan(f'../data/train/{idx}/')
scan_array = set_lungwin(get_pixels_hu(scans))

In [ ]:
import matplotlib.animation as animation

fig = plt.figure()

ims = []
for image in scan_array:
    im = plt.imshow(image, animated=True, cmap="Greys")
    plt.axis("off")
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=False,
                                repeat_delay=1000)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
idx = np.random.choice(train.Patient.unique())
train[train.Patient == idx]

In [ ]:
fig = plt.figure()
plt.plot(train.loc[train.Patient==idx, 'FVC'])

In [ ]:
scans = load_scan(f'../data/train/{idx}/')
scan_array = set_lungwin(get_pixels_hu(scans))

fig = plt.figure()

ims = []
for image in scan_array:
    im = plt.imshow(image, animated=True, cmap="Greys")
    plt.axis("off")
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=100, blit=False,
                                repeat_delay=1000)
HTML(ani.to_jshtml())

In [3]:
train, sub = read_df(cfg)
arimax = sm.tsa.ARIMA(endog=train['FVC'], exog=train[['Age','Male', 'Female', 'Ex-smoker', 'Never smoked', 'Currently smokes']], order=(1,0,1))

In [4]:
arimax.fit()

/home/giorgio/Desktop/Kaggle/kaggleEnv/lib/python3.6/site-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [6]:
arimax.geterrors()

TypeError: geterrors() missing 1 required positional argument: 'params'